# Koa核心原理

koa-compose巧妙利用函数调用栈，完成了栈结果，实现了洋葱模型；推荐通过调试，仔细观察每次入栈和出栈

In [2]:
// koa-compose源码 compose函数

function compose(middleware) {
  if (!Array.isArray(middleware)) throw new TypeError('Middleware stack must be an array!')
  for (const fn of middleware) {
    if (typeof fn !== 'function') throw new TypeError('Middleware must be composed of functions!')
  }

  /**
   * @param {Object} context
   * @return {Promise}
   * @api public
   */

  return function (context, next) {
    // last called middleware #
    let index = -1
    return dispatch(0)
    function dispatch(i) {
      if (i <= index) return Promise.reject(new Error('next() called multiple times'))
      index = i
      let fn = middleware[i]
      if (i === middleware.length) fn = next
      if (!fn) return Promise.resolve()
      try {
        return Promise.resolve(fn(context, dispatch.bind(null, i + 1)))
      } catch (err) {
        return Promise.reject(err)
      }
    }
  }
}

var middlewareList = [
  function (ctx, next) {
    console.log('1 - req', ctx)
    next()
    console.log('1 - res', ctx)
  },
  function (ctx, next) {
    console.log('2 - req', ctx)
    next()
  },
  async function (ctx, next) {
    await new Promise((resolve, reject) => {
      setTimeout(function () {
        var r = Math.random()
        if (r > 0.5) {
          resolve(r)
        } else {
          reject(r)
        }
      }, 250)
    }).then(() => {
      console.log('3 - req', ctx)
    })
    next()
    console.log('3 - res', ctx)
  },
]

var fn = compose(middlewareList)
fn()

1 - req undefined
2 - req undefined
1 - res undefined
3 - req undefined
3 - res undefined
